In [48]:
import pandas as pd
import os


def pprint(str_, f):
    print(str_)
    print(str_, end='\n', file=f)


def filter_data(filePath, threshold=10):
    data = []
    ratings = pd.read_csv(filePath, delimiter=",", encoding="latin1")
    ratings.columns = ['userId', 'itemId', 'Rating', 'timesteamp']

    rate_size_dic_i = ratings.groupby('itemId').size()
    # choosed_index_del_i = rate_size_dic_i.index[rate_size_dic_i < 10]
    choosed_index_del_i = rate_size_dic_i.index[rate_size_dic_i < threshold]
    ratings = ratings[~ratings['itemId'].isin(list(choosed_index_del_i))] # item freq more than 10

    user_unique = list(ratings['userId'].unique())
    movie_unique = list(ratings['itemId'].unique())

    u = len(user_unique)
    i = len(movie_unique)
    rating_num = len(ratings)
    return u, i, rating_num, user_unique, ratings


def get_min_group_size(ratings):
    rate_size_dic_u = ratings.groupby('userId').size()
    return min(rate_size_dic_u)


def my_reindex_data(ratings1, dic_u=None):
    data = []
    if dic_u is None:
        user_unique = list(ratings1['userId'].unique())
        user_index = list(range(0, len(user_unique)))
        dic_u = dict(zip(user_unique, user_index))
    movie_unique1 = list(ratings1['itemId'].unique())
    movie_index1 = list(range(0, len(movie_unique1)))
    dic_m1 = dict(zip(movie_unique1, movie_index1))
    for element in ratings1.values:
        data.append((dic_u[element[0]], dic_m1[element[1]], 1))
    data = sorted(data, key=lambda x: x[0])
    return data, dic_u

def reindex_data(ratings1, dic_u=None,dic_v=None):
    data = []
    if dic_u is None:
        user_unique = list(ratings1['userId'].unique())
        user_index = list(range(0, len(user_unique)))
        dic_u = dict(zip(user_unique, user_index))
    if dic_v is None:
        movie_unique1 = list(ratings1['itemId'].unique())
        movie_index1 = list(range(0, len(movie_unique1)))
        dic_v = dict(zip(movie_unique1, movie_index1))
    for element in ratings1.values:
        data.append((dic_u[element[0]], dic_v[element[1]], 1))
    data = sorted(data, key=lambda x: x[0])
    return data, dic_u


def get_common_data(data1, data2, common_user):
    rating_new_1 = data1[data1['userId'].isin(common_user)]
    rating_new_2 = data2[data2['userId'].isin(common_user)]
    return rating_new_1, rating_new_2


def get_unique_lenth(ratings):
    r_n = len(ratings)
    user_unique = list(ratings['userId'].unique())
    movie_unique = list(ratings['itemId'].unique())
    u = len(user_unique)
    i = len(movie_unique)
    return u, i, r_n


def filter_user(ratings1, ratings2):
    rate_size_dic_u1 = ratings1.groupby('userId').size()
    rate_size_dic_u2 = ratings2.groupby('userId').size()
    choosed_index_del_u1 = rate_size_dic_u1.index[rate_size_dic_u1 < 5]
    choosed_index_del_u2 = rate_size_dic_u2.index[rate_size_dic_u2 < 5]
    ratings1 = ratings1[~ratings1['userId'].isin(list(choosed_index_del_u1) + list(choosed_index_del_u2))]
    ratings2 = ratings2[~ratings2['userId'].isin(list(choosed_index_del_u1) + list(choosed_index_del_u2))]
    return ratings1, ratings2

def filter_item(ratings1, ratings2):
    rate_size_dic_u1 = ratings1.groupby('itemId').size()
    rate_size_dic_u2 = ratings2.groupby('itemId').size()
    choosed_index_del_u1 = rate_size_dic_u1.index[rate_size_dic_u1 < 5]
    choosed_index_del_u2 = rate_size_dic_u2.index[rate_size_dic_u2 < 5]
    ratings1 = ratings1[~ratings1['itemId'].isin(list(choosed_index_del_u1) + list(choosed_index_del_u2))]
    ratings2 = ratings2[~ratings2['itemId'].isin(list(choosed_index_del_u1) + list(choosed_index_del_u2))]
    return ratings1, ratings2


def write_to_txt(data, file):
    f = open(file, 'w+')
    for i in data:
        line = '\t'.join([str(x) for x in i]) + '\n'
        f.write(line)
    f.close


def get_common_user(data1, data2):
    common_user = list(set(data1).intersection(set(data2)))
    return len(common_user), common_user

def get_total_user(data1, data2):
    total_user = list(set(data1).union(set(data2)))
    return len(total_user), total_user


# cloth sport
# cell electronic
# game video
# cd movie
# music instrument

raw_path = '/home/hadh2/projects/cross_domain/cross_domain/CDRIB/dataset/raw_data'
data_name_s = 'movie'
data_name_t = 'video'
save_path = '/home/hadh2/projects/cross_domain/cross_domain/CDRIB/dataset/generated_data'

save_path_s = os.path.join(save_path, data_name_s + '_' + data_name_t)
save_path_t = os.path.join(save_path, data_name_t + '_' + data_name_s)
if not os.path.exists(save_path_s):
    os.makedirs(save_path_s)
if not os.path.exists(save_path_t):
    os.makedirs(save_path_t)

data_dic = {'sport': 'ratings_Sports_and_Outdoors', 'electronic': 'ratings_Electronics',
            'cloth': 'ratings_Clothing_Shoes_and_Jewelry', 'cell': 'ratings_Cell_Phones_and_Accessories',
            'instrument': 'ratings_Musical_Instruments', 'game': 'ratings_Toys_and_Games', 'video': 'ratings_Video_Games',
            'book':'ratings_Books', 'movie':'ratings_Movies_and_TV', 'cd': 'ratings_CDs_and_Vinyl', 'music':'ratings_Digital_Music'}


filepath1 = os.path.join(raw_path, data_dic[data_name_s] + '.csv')
filepath2 = os.path.join(raw_path, data_dic[data_name_t] + '.csv')
save_file1 = os.path.join(save_path_s, 'common_new_reindex.txt')
save_file2 = os.path.join(save_path_t, 'common_new_reindex.txt')
save_file3 = os.path.join(save_path_s, 'new_reindex.txt')
save_file4 = os.path.join(save_path_t, 'new_reindex.txt')
source_info_path = os.path.join(save_path_s, '%s_%s_data_info.txt' % (data_name_s, data_name_t))
target_info_path = os.path.join(save_path_t, '%s_%s_data_info.txt' % (data_name_t, data_name_s))
source_info_file = open(source_info_path, 'w+')
target_info_file = open(target_info_path, 'w+')
u_num, i_num, r_num, user_unique, data = filter_data(filepath1) # item<10
u_num2, i_num2, r_num2, user_unique2, data2 = filter_data(filepath2) # item<10


data, data2 = filter_user(data, data2) # del overlap user < 5
data, data2 = filter_item(data, data2) # del overlap item < 5
user_unique = list(data['userId'].unique())
user_unique2 = list(data2['userId'].unique())
item_unique = list(data['itemId'].unique())
item_unique2 = list(data2['itemId'].unique())



c_n, common_user = get_common_user(user_unique, user_unique2)
t_n, total_user = get_total_user(user_unique, user_unique2)
pprint('raw_data 1 info : %d %d %d' % (u_num, i_num, r_num), source_info_file)
pprint('raw_data 2 info : %d %d %d' % (u_num2, i_num2, r_num2), source_info_file)
pprint('common user num %d' % c_n, source_info_file)
pprint('total user num %d' % t_n, source_info_file)


new_data_1, new_data_2 = get_common_data(data, data2, common_user) # overlap


u, i, r = get_unique_lenth(data)
u2, i2, r2 = get_unique_lenth(data2)
pprint('after filter_data1 info : %d %d %d %.6f' % (u, i, r, r / (u * i)), source_info_file)
pprint('after filter_data2 info : %d %d %d %.6f' % (u2, i2, r2, r2 / (u2 * i2)), source_info_file)

u, i, r = get_unique_lenth(new_data_1)
u2, i2, r2 = get_unique_lenth(new_data_2)
pprint('after common_data1 info : %d %d %d %.6f' % (u, i, r, r / (u * i)), source_info_file)
pprint('after common_data2 info : %d %d %d %.6f' % (u2, i2, r2, r2 / (u2 * i2)), source_info_file)


user_index = list(range(0, len(total_user)))
dic_u = dict(zip(total_user, user_index))

item_index1 = list(range(0, len(item_unique)))
dic_v1 = dict(zip(item_unique, item_index1))

item_index2 = list(range(0, len(item_unique2)))
dic_v2 = dict(zip(item_unique2, item_index2))

common_data1, dic_u = reindex_data(new_data_1,dic_u,dic_v1)
common_data2, dic_u2 = reindex_data(new_data_2, dic_u,dic_v2)
write_to_txt(common_data1, save_file1)
write_to_txt(common_data2, save_file2)

data1, dic_u = reindex_data(data, dic_u, dic_v1)
data2, dic_u2 = reindex_data(data2, dic_u, dic_v2)
write_to_txt(data1, save_file3)
write_to_txt(data2, save_file4)

pprint('write data finished!', source_info_file)


raw_data 1 info : 1956059 53945 4195275
raw_data 2 info : 780447 17971 1223805
common user num 6011
total user num 118980
after filter_data1 info : 106522 41995 1385458 0.000310
after filter_data2 info : 18469 8866 163874 0.001001
after common_data1 info : 6011 28855 179185 0.001033
after common_data2 info : 6011 8706 75944 0.001451
write data finished!


In [54]:
import codecs
import copy
import os
import random


def read_dataset(file_name):
    data = {}
    with open(file_name,"r",encoding="utf-8") as fr:
        for line in fr:
            user, item, score = line.strip().split("\t")
            if user not in data.keys():
                data[user] = [item]
            else :
                data[user].append(item)
    return data

def create_user_dict(user, data):
    user = copy.deepcopy(user)
    for u in data.keys():
        if u not in user.keys():
            user[u] = len(user)
    return user

def create_item_dict(item, data):
    for u in data.keys():
        for i in data[u]:
            if i not in item.keys():
                item[i] = len(item)
    return item


def ensure_dir(d, verbose=True):
    if not os.path.exists(d):
        if verbose:
            print("Directory {} do not exist; creating...".format(d))
        os.makedirs(d)

def generate_train_valid_test(file, total, common, user, item, choose):
    train_file = file + "train.txt"
    valid_file = file + "valid.txt"
    test_file = file + "test.txt"

    #source_total_data, source_common_data, source_user, source_item, 0
    ## total: dict user total: list[item]
    ## common: dict user common: list[item]
    ## user: user mapping dict from index in file reindex.txt to index of that domain only
    ## item: same
    ## chose: 0 is source data, 1 is target data
    itemx = 0

    L = int(len(common) * 0.1)
    LL = L * 9
    LLL = L * 8

    train_number = 0
    valid_number = 0
    test_number = 0
    with codecs.open(train_file,"w",encoding="utf-8") as fw:
        with codecs.open(test_file, "w", encoding="utf-8") as fw2:
            with codecs.open(valid_file, "w", encoding="utf-8") as fw3:
                for da in total: #da is the old user index from file reindex.txt
                    if choose == 0 and user[da] >= LL and user[da] < len(common): 
                        #10% last user of common user is used as valid or test data (random)
                        for i in total[da]: # total[da] is the item list of old user index
                            itemx = max(itemx, int(item[i]))
                            if random.randint(0,1):
                                test_number+=1
                                fw2.write(str(user[da]) + "\t" + str(item[i]) + "\n") # cold-start user for test
                            else :
                                valid_number+=1
                                fw3.write(str(user[da]) + "\t" + str(item[i]) + "\n")  # cold-start user for valid
                    elif choose == 1 and user[da] >= LLL and user[da] < LL:
                        for i in total[da]:
                            itemx = max(itemx, int(item[i]))
                            if random.randint(0, 1):
                                test_number += 1
                                fw2.write(str(user[da]) + "\t" + str(item[i]) + "\n")  # cold-start user for test
                            else:
                                valid_number += 1
                                fw3.write(str(user[da]) + "\t" + str(item[i]) + "\n")  # cold-start user for valid
                    else:
                        for i in total[da]:
                            train_number += 1
                            itemx = max(itemx, int(item[i]))
                            fw.write(str(user[da])+"\t"+str(item[i])+"\n")

    print("Train {}, valid {}, test {}".format(train_number, valid_number, test_number))
    print("sss", itemx)


source = "movie"
target = "video"
f1 = '/home/hadh2/projects/cross_domain/cross_domain/CDRIB/dataset/generated_data' + '/' + source + "_" + target + "/"
f2 = '/home/hadh2/projects/cross_domain/cross_domain/CDRIB/dataset/generated_data' + '/' + target + "_" + source + "/"

source_common_data = read_dataset(f1 + "common_new_reindex.txt") # same users data
target_common_data = read_dataset(f2 + "common_new_reindex.txt")

user_dict = {} # re-index
source_item = {}
target_item = {}
if len(source_common_data) == len(target_common_data):
    user_dict = create_user_dict(user_dict, source_common_data)
else:
    print("error!!!!!!")
    exit(0)

source_total_data = read_dataset(f1 + "new_reindex.txt")
target_total_data = read_dataset(f2 + "new_reindex.txt")

source_user = create_user_dict(user_dict, source_total_data) # re-index
target_user = create_user_dict(user_dict, target_total_data)

source_item = create_item_dict(source_item, source_total_data)
target_item = create_item_dict(target_item, target_total_data)

print(len(source_user))
print(len(target_user))
print(len(source_item))
print(len(target_item))

generate_train_valid_test(f1, source_total_data, source_common_data, source_user, source_item, 0)
generate_train_valid_test(f2, target_total_data, target_common_data, target_user, target_item, 1)



106522
18469
41995
8866
Train 1364788, valid 10363, test 10307
sss 41994
Train 155867, valid 3962, test 4045
sss 8865
